In [27]:
from parsing import parser
from parsing_ast import transformer
import parsing_ast

In [324]:
input_str = """ 
var a = True;
if(a) {

}
"""

In [325]:
tree = parser.parse(input_str)
ast = transformer.transform(tree)

DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:IDENTIFIER - True
DEBUG:parsing_ast:expression - [identifier('True')]
DEBUG:parsing_ast:Processing Variable Declaration with [declaration_variable('var'), identifier('a'), Token('ASSIGNMENT_OPERATOR', '='), identifier('True'), Token('END_OF_STATEMENT', ';')]
DEBUG:parsing_ast:Variable Declaration - var, [identifier('a')], [identifier('True')]
DEBUG:parsing_ast:Processing 'declare' with [declaration(declaration_variable('var'), [identifier('a')], [identifier('True')])]
DEBUG:parsing_ast:Processing 'statement' with [declaration(declaration_variable('var'), [identifier('a')], [identifier('True')])]
DEBUG:parsing_ast:IDENTIFIER - a
DEBUG:parsing_ast:expression - [identifier('a')]
DEBUG:parsing_ast:Processing Block with [Token('CURLY_OPEN', '{'), Token('CURLY_CLOSE', '}')]
DEBUG:parsing_ast:Processing Else Temp with []
DEBUG:parsing_ast:Processing Control with [if_else(identifier('a'), block(None), None)]
DEBUG:parsing_ast:Processing 'state

['var', 'a', '=', 'True', ';', 'if', '(', 'a', ')', '{', '}']


In [326]:
ast

statements([declaration(declaration_variable('var'), [identifier('a')], [identifier('True')]), if_else(identifier('a'), block(None), None)])

In [327]:
class Symbol(object):
    def __init__(self, name, type=None, category=None) -> None:
        self.name = name
        self.type = type
        self.category = category

In [328]:
class BuiltinTypeSymbol(Symbol):
    def __init__(self, name) -> None:
        super().__init__(name)

    def __str__(self) -> str:
        return self.name
    
    def __repr__(self) -> str:
        return "<{class_name}(name='{name}')>".format(
            class_name=self.__class__.__name__,
            name=self.name,
        )

In [311]:
class VarSymbol(Symbol):
    def __init__(self, name, type=None, category=None) -> None:
        super().__init__(name, type, category)
    
    def __str__(self) -> str:
        return "<{class_name}(name='{name}', type='{type}', category='{category}')>".format(
            class_name=self.__class__.__name__,
            name=self.name,
            type=self.type,
            category=self.category
        )
    
    __repr__ = __str__

In [312]:
class SymbolTable(object):
    def __init__(self) -> None:
        self._symbols = {}
        self._init_builtins()

    def _init_builtins(self) -> None:
        self.insert(BuiltinTypeSymbol('INTEGER'))
        self.insert(BuiltinTypeSymbol('REAL'))
        self.insert(BuiltinTypeSymbol('STRING'))
        self.insert(BuiltinTypeSymbol('BOOLEAN'))
        self.insert(BuiltinTypeSymbol('ARRAY'))
        self.insert(BuiltinTypeSymbol('TUPLE'))
        self.insert(BuiltinTypeSymbol('LIST'))

    def __str__(self) -> str:
        symtab_header = 'Symbol table contents'
        lines = ['\n', symtab_header, '_' * len(symtab_header)]
        lines.extend(
            ('%7s: %r' % (key, value))
            for key, value in self._symbols.items()
        )
        lines.append('\n')
        s = '\n'.join(lines)
        return s
    
    __repr__ = __str__

    def insert(self, symbol) -> None:
        print('Insert: %s' % symbol.name)
        self._symbols[symbol.name] = symbol

    def lookup(self, name) -> Symbol:
        print('Lookup: %s' % name)
        return self._symbols.get(name, None)

In [383]:
input_str = """ 
/* Function call */
/* Function definition */
func add(var x, var y) {
}
var result = add(3, 4);
"""
tree = parser.parse(input_str)
ast = transformer.transform(tree)

DEBUG:parsing_ast:IDENTIFIER - add
DEBUG:parsing_ast:IDENTIFIER - x
DEBUG:parsing_ast:Processing Parameter with [declaration_variable('var'), identifier('x')]
DEBUG:parsing_ast:IDENTIFIER - y
DEBUG:parsing_ast:Processing Parameter with [declaration_variable('var'), identifier('y')]
DEBUG:parsing_ast:Processing Parameters with [parameter(declaration_variable('var'), identifier('x')), Token('COMMA', ','), parameter(declaration_variable('var'), identifier('y'))]
DEBUG:parsing_ast:Processing Block with [Token('CURLY_OPEN', '{'), Token('CURLY_CLOSE', '}')]
DEBUG:parsing_ast:Processing Function with [Token('FUNCTION_DECLARATION', 'func'), identifier('add'), Token('ROUND_OPEN', '('), parameters([parameter(declaration_variable('var'), identifier('x')), parameter(declaration_variable('var'), identifier('y'))]), Token('ROUND_CLOSE', ')'), block(None)]
DEBUG:parsing_ast:Processing Control with [function(identifier('add'), parameters([parameter(declaration_variable('var'), identifier('x')), parame

['func', 'add', '(', 'var', 'x', ',', 'var', 'y', ')', '{', '}', 'var', 'result', '=', 'add', '(', '3', ',', '4', ')', ';']


In [387]:
symbol_table = SymbolTable()

Insert: INTEGER
Insert: REAL
Insert: STRING
Insert: BOOLEAN
Insert: ARRAY
Insert: TUPLE


In [388]:
def traverse_ast(node, indent="", last=True):
    if isinstance(node, list):
        for idx, item in enumerate(node):
            if idx == len(node) - 1:
                traverse_ast(item, indent, last=True)
            else:
                traverse_ast(item, indent, last=False)
        return
    print(type(node))
    if type(node) == parsing_ast.Function:
        if symbol_table.lookup(node.name.name):
            raise Exception('Error: Duplicate Function Declaration: %s' % node.name.name)
        symbol_table.insert(VarSymbol(node.name.name, 'function'))
    if isinstance(node, parsing_ast.InbuiltFunctionCall):
        if not symbol_table.lookup(node.base.name):
            raise Exception('Error: Symbol not found: %s' % node.base.name)
    if type(node) == parsing_ast.FunctionCall:
        if not symbol_table.lookup(node.name.name):
            raise Exception('Error: Symbol not found: %s' % node.name.name)
    if isinstance(node, parsing_ast.VariableDeclaration):
        for name in node.identifier:
            if symbol_table.lookup(name.name):
                raise Exception('Error: Duplicate identifier found: %s' % name.name)
            symbol_table.insert(VarSymbol(name.name, node.declaration_type.value))
    if isinstance(node, parsing_ast.Declaration):
        if symbol_table.lookup(node.identifier.name):
            raise Exception('Error: Duplicate identifier found: %s' % node.identifier.name)
        symbol_table.insert(VarSymbol(node.identifier.name, node.declaration_type.value))
    if isinstance(node, parsing_ast.Assignment):
        if not symbol_table.lookup(node.identifier.name):
            raise Exception('Error: Symbol not found: %s' % node.identifier.name)
    if type(node) == parsing_ast.FunctionArgumentList:
        for arg in node.arguments:
            if isinstance(arg, parsing_ast.Identifier) and not symbol_table.lookup(arg.name):
                raise Exception('Error: Symbol not found: %s' % arg.name)
    if type(node) == parsing_ast.PrintArgumentList:
        for arg in node.arguments:
            if isinstance(arg, parsing_ast.Identifier) and not symbol_table.lookup(arg.name):
                raise Exception('Error: Symbol not found: %s' % arg.name)
    if type(node) == parsing_ast.IfElse:
        if isinstance(node.expression, parsing_ast.Identifier) and not symbol_table.lookup(node.expression.name):
            raise Exception('Error: Symbol not found: %s' % node.expression.name)
    if type(node) == parsing_ast.While:
        if isinstance(node.expression, parsing_ast.Identifier) and not symbol_table.lookup(node.expression.name):
            raise Exception('Error: Symbol not found: %s' % node.expression.name)
    if type(node) == parsing_ast.DoWhile:
        if isinstance(node.expression, parsing_ast.Identifier) and not symbol_table.lookup(node.expression.name):
            raise Exception('Error: Symbol not found: %s' % node.expression.name)
    if type(node) == parsing_ast.BinaryOp:
        if isinstance(node.children[0], parsing_ast.Identifier) and not symbol_table.lookup(node.children[0].name):
            raise Exception('Error: Symbol not found: %s' % node.children[0].name)
        if isinstance(node.children[1], parsing_ast.Identifier) and not symbol_table.lookup(node.children[1].name):
            raise Exception('Error: Symbol not found: %s' % node.children[1].name)
    if type(node) == parsing_ast.UnaryExpression:
        if isinstance(node.children[0], parsing_ast.Identifier) and not symbol_table.lookup(node.children[0].name):
            raise Exception('Error: Symbol not found: %s' % node.children[0].name)
        if isinstance(node.operator, parsing_ast.Identifier) and not symbol_table.lookup(node.operator.name):
            raise Exception('Error: Symbol not found: %s' % node.operator.name)
    if not hasattr(node, '__dict__'):
        return 
    children = [(k, v) for k, v in node.__dict__.items() if not k.startswith('_') and v]
    for idx, (attr, child) in enumerate(children):
        is_last = idx == len(children) - 1
        traverse_ast(child, last=is_last)

In [389]:
traverse_ast(ast)

<class 'parsing_ast.Statements'>
<class 'parsing_ast.Function'>
Lookup: add
Insert: add
<class 'parsing_ast.Identifier'>
<class 'str'>
<class 'parsing_ast.Parameters'>
<class 'parsing_ast.Parameter'>
<class 'parsing_ast.DeclarationVariable'>
<class 'str'>
<class 'parsing_ast.Identifier'>
<class 'str'>
<class 'parsing_ast.Parameter'>
<class 'parsing_ast.DeclarationVariable'>
<class 'str'>
<class 'parsing_ast.Identifier'>
<class 'str'>
<class 'parsing_ast.Block'>
<class 'parsing_ast.VariableDeclaration'>
Lookup: result
Insert: result
<class 'parsing_ast.DeclarationVariable'>
<class 'str'>
<class 'parsing_ast.Identifier'>
<class 'str'>
<class 'parsing_ast.FunctionCall'>
Lookup: add
<class 'parsing_ast.Identifier'>
<class 'str'>
<class 'parsing_ast.FunctionArgumentList'>
<class 'parsing_ast.Literal'>
<class 'str'>
<class 'parsing_ast.Literal'>
<class 'str'>


In [337]:
symbol = symbol_table.lookup('i')
symbol

Lookup: i


<VarSymbol(name='i', type='var', category='None')>

In [355]:
symbol_table._symbols

{'INTEGER': <BuiltinTypeSymbol(name='INTEGER')>,
 'REAL': <BuiltinTypeSymbol(name='REAL')>,
 'STRING': <BuiltinTypeSymbol(name='STRING')>,
 'BOOLEAN': <BuiltinTypeSymbol(name='BOOLEAN')>,
 'ARRAY': <BuiltinTypeSymbol(name='ARRAY')>,
 'TUPLE': <BuiltinTypeSymbol(name='TUPLE')>,
 'i': <VarSymbol(name='i', type='var', category='None')>}

In [158]:
def a():
    print('a')

a = 5